In [12]:
from src.Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.PureSVDRecommender import ScaledPureSVDRecommender
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDItemRecommender
from src.Recommenders.Neural.MultVAE_PyTorch_Recommender import MultVAERecommender_PyTorch
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender



import numpy as np
import random
import os


SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)


import pandas as pd
import scipy.sparse as sps

dataframe_items=pd.read_csv("Data/data_ICM_metadata.csv", low_memory=True)

icm=sps.csr_matrix((dataframe_items["data"].values,(dataframe_items["item_id"].values.astype(int),dataframe_items["feature_id"].values)))

dataframe_users = pd.read_csv("Data/data_target_users_test.csv", low_memory=True)
dataframe_urm=pd.read_csv("Data/data_train.csv", low_memory=True)

index_id,original_id=pd.factorize(dataframe_urm["user_id"].unique())

map_original_to_index=pd.Series(index_id,index=original_id)
dataframe_urm["user_id"]=dataframe_urm["user_id"].map(map_original_to_index)

urm=sps.csr_matrix((dataframe_urm["data"].values,(dataframe_urm["user_id"].values.astype(int),dataframe_urm["item_id"].values)))

In [13]:
urm_stacked=sps.vstack([urm,icm.T])
slim_recommender=SLIMElasticNetRecommender(urm)
alpha=0.76679688335902
urm_slim_stacked=sps.vstack([urm*alpha,icm.T*(1-alpha)])
slim_stacked_recommender=SLIMElasticNetRecommender(urm_slim_stacked)
pure_svd_recommender=PureSVDItemRecommender(urm)
rp3beta_recommender=RP3betaRecommender(urm)
rp3beta_stacked_recommender=RP3betaRecommender(urm_stacked)
scaled_svd_recommender= ScaledPureSVDRecommender(urm)
multvae_recommender=MultVAERecommender_PyTorch(urm)

In [14]:
folder_path='trained_models/sub/'
slim_recommender.load_model(folder_path=folder_path,file_name='best_sub_slim.zip')
slim_stacked_recommender.load_model(folder_path=folder_path,file_name='best_sub_stacked_slim.zip')
pure_svd_recommender.load_model(folder_path=folder_path,file_name='best_sub_pure_svd.zip')
scaled_svd_recommender.load_model(folder_path=folder_path,file_name='best_sub_scaled_svd.zip')
rp3beta_recommender.load_model(folder_path=folder_path,file_name='best_sub_rp3beta.zip')
rp3beta_stacked_recommender.load_model(folder_path=folder_path,file_name='best_sub_stacked_rp3beta.zip')
multvae_recommender.load_model(folder_path=folder_path,file_name='best_sub_MultVAE.zip')

SLIMElasticNetRecommender: Loading model from file 'trained_models/sub/best_sub_slim.zip'
SLIMElasticNetRecommender: Loading complete
SLIMElasticNetRecommender: Loading model from file 'trained_models/sub/best_sub_stacked_slim.zip'
SLIMElasticNetRecommender: Loading complete
PureSVDItemRecommender: Loading model from file 'trained_models/sub/best_sub_pure_svd.zip'
PureSVDItemRecommender: Loading complete
ScaledPureSVDRecommender: Loading model from file 'trained_models/sub/best_sub_scaled_svd.zip'
ScaledPureSVDRecommender: Loading complete
RP3betaRecommender: Loading model from file 'trained_models/sub/best_sub_rp3beta.zip'
RP3betaRecommender: Loading complete
RP3betaRecommender: Loading model from file 'trained_models/sub/best_sub_stacked_rp3beta.zip'
RP3betaRecommender: Loading complete


In [18]:
import optuna
database_path='hypertuning_results/optuna_hybrid_linear_best.db'
study=optuna.load_study(study_name='hyperparameters_hybrid_linear',storage=f'sqlite:///{database_path}')
best_params=study.best_params

In [19]:
weights = [value for value in best_params.values() if isinstance(value, (int, float))]

In [20]:
recommenders_list={}

recommenders_list['slim']=slim_recommender
recommenders_list['slim_stacked']=slim_stacked_recommender
recommenders_list['pure_svd']=pure_svd_recommender
recommenders_list['scaled_svd']=scaled_svd_recommender
recommenders_list['rp3beta']=rp3beta_recommender
recommenders_list['rp3beta_stacked']=rp3beta_stacked_recommender
recommenders_list['multvae']=multvae_recommender
recommenders=[
recommenders_list['slim'],
recommenders_list['pure_svd'],
recommenders_list['scaled_svd'],
recommenders_list['rp3beta'],
recommenders_list['rp3beta_stacked'],
recommenders_list['multvae']
]

In [21]:
from src.Recommenders.Hybrids.LinearWeightedRecommender import LinearWeightedRecommender
hybrid_recommender=LinearWeightedRecommender(urm,recommenders,weights)

In [ ]:
import os
import csv
import time

output_folder_path = "submissions/"
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

user_id = dataframe_users['user_id']
recommendations = []
startTime = time.time()
filename="result_linear_hybrid_final_best.csv"
file_path = os.path.join(output_folder_path, filename)
with open(file_path ,mode='w',newline='') as file:
    writer=csv.writer(file)
    writer.writerow(["user_id","item_list"])
    for user in user_id:
        writer.writerow([user," ".join(map(str,hybrid_recommender.recommend(map_original_to_index[user], cutoff=10)))])
        if user%1000 == 0:
            print("elapsed: {} current user: {}".format(time.time()-startTime,user))